In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from gPhoton.pipeline import execute_pipeline

Notebook summary: Final-ish implementation of catalog generation. Note that a lot of functions have been moved into the external modules imported just below.

Pipeline steps:
1. ~Run photometry with extraction on NUV~
2. ~Run photometry with extraction on FUV~
3. ~Run photometry on FUV with NUV source positions~
4. ~Run photometry on NUV with FUV source positions~
5. ~Generate a merged catalog from NUV extractions~
6. ~Generate a merged catalog from FUV extractions~
7. Generate a merged catalog from all NUV & FUV detections

In [2]:
from glcat_merge_utils import *
from lightcurve_utils import *
from glcat_catalog_utils import *

In [3]:
data = accumulate_run_data(23456)
ncat = make_catalog(data,'NUV')
fcat = make_catalog(data,'FUV')
cats = {'NUV':ncat,'FUV':fcat}

In [4]:
def crossmatch_catalogs(match_table, master_table, # match_table to master_table
                        match_radius=2.5, # arcseconds
                       ):
    master_catalog = SkyCoord(ra=master_table['RA'].values*u.degree,
                              dec=master_table['DEC'].values*u.degree)
    match_catalog = SkyCoord(ra=match_table['RA'].values*u.degree,
                             dec=match_table['DEC'].values*u.degree)
    catalog_ix, d2d, d3d = match_catalog.match_to_catalog_sky(master_catalog)
    sep_constraint = d2d < match_radius * u.arcsec

    return catalog_ix,d2d

In [5]:
def xmatch(cats,
          bands = ('FUV','NUV'), # maps FUV -> NUV
          ):
    b0,b1 = bands
    mstr = f"{bands[0][0]}2{bands[1][0]}"

    idx,d2d=crossmatch_catalogs(cats[b0],cats[b1])
    match = pd.DataFrame({'idx':idx,'d2d':d2d,
                          'good_match':np.array(d2d <= 2.5 * u.arcsec,dtype=int),
                          'best_match':np.zeros(len(idx),dtype=int),
                          'n_matches':np.zeros(len(idx),dtype=int),
                          'n_good_matches':np.zeros(len(idx),dtype=int)})
    for ix in np.unique(idx):
        best_ix = np.argmin(match.loc[match['idx']==ix]['d2d'].values)
        match.loc[match.loc[match['idx']==ix].iloc[best_ix].name,'best_match']=1
        match.loc[match['idx']==ix,'n_matches']=len(match.loc[match['idx']==ix])
        match.loc[match['idx']==ix,'n_good_matches']=len(match.loc[match['idx']==ix].loc[match['good_match']==1])
    
    # update the FUV catalog with the matches to the NUV catalog
    cats[b0][f'{mstr}_MATCH_INDEX']=np.full(len(cats[b0]),-1)
    cats[b0].loc[match.index,f'{mstr}_NUV_MATCH_INDEX']=np.array(match['idx'].values,dtype=int)

    cats[b0][f'{mstr}_MATCH_DISTANCE']=np.full(len(cats[b0]),np.nan)
    cats[b0].loc[match.index,f'{mstr}_MATCH_DISTANCE']=match['d2d'].values

    cats[b0][f'{mstr}_GOOD_MATCH_FLAG']=np.full(len(cats[b0]),-1)
    cats[b0].loc[match.index,f'{mstr}_GOOD_MATCH_FLAG']=np.array(match['good_match'].values,dtype=int)

    cats[b0][f'{mstr}_BEST_MATCH_FLAG']=np.full(len(cats[b0]),-1)
    cats[b0].loc[match.index,f'{mstr}_BEST_MATCH_FLAG']=np.array(match['best_match'].values,dtype=int)
    
    # update the NUV catalog with the reverse lookup to the "best" matches in the FUV catalog
    #    
    # the index in the FUV catalog of the nearest match aka "best match"
    cats[b1][f'{mstr}_BEST_MATCH_INDEX']=np.full(len(cats[b1]),-1)
    cats[b1].loc[match['idx'][match['best_match']==1],
                f'{mstr}_BEST_MATCH_INDEX']=np.array(
                    match.loc[match['best_match']==1]['idx'].values,dtype=int)
    # this distance in decimal degress (?) to the nearest match aka "best match"
    cats[b1][f'{mstr}_BEST_MATCH_DISTANCE']=np.full(len(cats[b1]),np.nan)
    cats[b1].loc[match['idx'][match['best_match']==1],
                f'{mstr}_BEST_MATCH_DISTANCE']=np.array(
                    match.loc[match['best_match']==1]['d2d'].values)
    # it is possible that the "best match" is not a "good match" in which case this will be zero
    cats[b1][f'{mstr}_GOOD_MATCH_COUNT']=np.full(len(cats[b1]),-1)
    cats[b1].loc[match['idx'][match['best_match']==1],
                f'{mstr}_GOOD_MATCH_COUNT']=np.array(
                    match.loc[match['best_match']==1]['n_good_matches'].values)
    return {'NUV':ncat,'FUV':fcat}
%time cats=xmatch(cats,bands=('FUV','NUV'))
%time cats=xmatch(cats,bands=('NUV','FUV'))

CPU times: user 3.9 s, sys: 48.1 ms, total: 3.95 s
Wall time: 4 s
CPU times: user 4.01 s, sys: 11 ms, total: 4.03 s
Wall time: 4.03 s


In [6]:
cats['NUV']

,ECLIPSE,RA,DEC,NUV_XCENTER,NUV_YCENTER,NUV_SUM_APER0,NUV_EDGE_APER0,NUV_MASK_APER0,NUV_CPS_APER0,NUV_CPS_ERR_APER0,...,FUV_EDGE_FLAG_APER6,FUV_EXPT,F2N_BEST_MATCH_INDEX,F2N_BEST_MATCH_DISTANCE,F2N_GOOD_MATCH_COUNT,N2F_MATCH_INDEX,N2F_NUV_MATCH_INDEX,N2F_MATCH_DISTANCE,N2F_GOOD_MATCH_FLAG,N2F_BEST_MATCH_FLAG
0,23456,323.448811,-2.638534,1638.452026,170.996174,74.195186,1,0,0.049456,0.005742,...,1,1678.19451,-1,NaN,-1,-1,1.0,0.014035,0,0
1,23456,323.458572,-2.636944,1615.050994,174.814059,35.807291,0,0,0.023868,0.003989,...,1,1678.19451,-1,NaN,-1,-1,1.0,0.010425,0,0
2,23456,323.448619,-2.635363,1638.913971,178.608991,114.462236,0,0,0.076297,0.007131,...,1,1678.19451,-1,NaN,-1,-1,1.0,0.011628,0,0
3,23456,323.509375,-2.632623,1493.247071,185.185578,50.258974,0,0,0.033501,0.004726,...,0,1678.19451,3,0.000732,0,-1,0.0,0.000732,0,1
4,23456,323.459455,-2.632761,1612.932867,184.853847,33.605723,0,0,0.022400,0.003864,...,0,1678.19451,4,0.006737,0,-1,1.0,0.006737,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835,23456,323.546199,-1.505455,1404.865650,2890.475707,63.903954,0,0,0.042596,0.005329,...,0,1678.19451,-1,NaN,-1,-1,3453.0,0.008293,0,0
4836,23456,323.548553,-1.504981,1399.217901,2891.612115,80.647389,0,1,0.053757,0.005986,...,0,1678.19451,-1,NaN,-1,-1,3453.0,0.005895,0,0
4837,23456,323.543522,-1.505130,1411.288694,2891.256176,47.527211,0,0,0.031680,0.004595,...,0,1678.19451,-1,NaN,-1,-1,3454.0,0.010413,0,0
4838,23456,323.541941,-1.501864,1415.081510,2899.095427,51.393869,0,1,0.034257,0.004779,...,0,1678.19451,-1,NaN,-1,-1,3454.0,0.009242,0,0


There are three catalog tables generated per visit. The band-specific tables contain one entry / row per source detected in the visit-level image corresponding to the band of that table. So the NUV table (or `ncat`) contains one entry / row per source detected by DAOphot (via gPhoton2) in the visit-level NUV image. It then contains summary statistis for band-specific photometry as well as summary statistics for alternate-band photometry, when that band is valid, using identical sky coordinates for extraction on the alternate-band image. Another way to say this is that it contains "forced photometry" on the alternate band image. So the `ncat` contains photometry for NUV as well as FUV on the same source positions determined from the NUV image. ANd the `fcat` contains photometry for FUV as well as NUV on the same soure positions determined from the FUV image. Each band-specific table also contains a set of columns that describe the crossmatch between the `ncat` and `fcat`. The current columns are as follows.

| COLUMN NAME | DEFINITION | NOTES |
|-------------|------------|-------|
| `ECLIPSE` | The GALEX eclipse number of the observation. |
| `LEG` | **NOT YET IMPLEMENTED**
| `RA` / `DEC` | Right Ascension and Declination in J2000 decimal degrees at the center of the photometric aperture. | This is the location of a source detection as determined by DAOphot as implemented in the gPhoton2 software. |
| [NF]UV_[XY]CENTER | The x and y pixel coordinate in the corresponding band's image at the center of the photometric aperture. | The images have accurate WCS information, so they can be aligned in sky coordinates, but they do not generally (never?) have the same center coordinate or dimensions and therefore are not pre-aligned in image coordinates |
| [NF]UV_SUM_APER[0-6] | The sum of response-weighted counts inside of the photometric aperture in the particular band. | The `APER[0-6]` is a mapping to a specific set of photometric aperture radii that were used in the original mission-produced photometic catalogs. The mapping and related information appears in a table below. |
| [NF]UV_EDGE_APER[0-6] | Redundant w/ `MASK_FLAG` below?  |
| [NF]UV_MASK_APER[0-6] | Redundant w/ `EDGE_FLAG` below?  |
| [NF]UV_CPS_APER[0-6] |   |
| [NF]UV_CPS_ERR_APER[0-6] |   |
| [NF]UV_MAG_APER[0-6] |   |
| [NF]UV_MAG_ERR_APER[0-6] |   |
| [NF]UV_MASK_FLAG_APER[0-6] |   |
| [NF]UV_EDGE_FLAG_APER[0-6] |   |
| [NF]UV_EXPT |   |
| [NF]UV_EXTENDED |   |
| ECLIPSE | Redundant.  |
| [FN]2[NF]_BEST_MATCH_INDEX |   |
| [FN]2[NF]_BEST_MATCH_DISTANCE |   |
| [FN]2[NF]_GOOD_MATCH_COUNT |   |
| [NF]2[FN]_MATCH_INDEX |   |
| [NF]2[FN]_NUV_MATCH_INDEX |   |
| [NF]2[FN]_MATCH_DISTANCE |   |
| [NF]2[FN]_GOOD_MATCH_FLAG |   |
| [NF]2[FN]_BEST_MATCH_FLAG |   |

[1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3]
| APER NUMBER | RADIUS (as) | CORRECTION (AB Mag) |
|-------------|------------|-------|
| 0 | 1.5 | foo |
| 1 | 2.3 |  |
| 2 | 3.8 |  |
| 3 | 6.0 |  |
| 4 | 9.0 |  |
| 5 | 12.8 |  |
| 6 | 17.3 |  |


In [11]:
for a in list(cats['NUV'].keys()):
    print(f'| {a} |   |')

| ECLIPSE |   |
| RA |   |
| DEC |   |
| NUV_XCENTER |   |
| NUV_YCENTER |   |
| NUV_SUM_APER0 |   |
| NUV_EDGE_APER0 |   |
| NUV_MASK_APER0 |   |
| NUV_CPS_APER0 |   |
| NUV_CPS_ERR_APER0 |   |
| NUV_MAG_APER0 |   |
| NUV_MAG_ERR_APER0 |   |
| NUV_MASK_FLAG_APER0 |   |
| NUV_EDGE_FLAG_APER0 |   |
| NUV_SUM_APER1 |   |
| NUV_EDGE_APER1 |   |
| NUV_MASK_APER1 |   |
| NUV_CPS_APER1 |   |
| NUV_CPS_ERR_APER1 |   |
| NUV_MAG_APER1 |   |
| NUV_MAG_ERR_APER1 |   |
| NUV_MASK_FLAG_APER1 |   |
| NUV_EDGE_FLAG_APER1 |   |
| NUV_SUM_APER2 |   |
| NUV_EDGE_APER2 |   |
| NUV_MASK_APER2 |   |
| NUV_CPS_APER2 |   |
| NUV_CPS_ERR_APER2 |   |
| NUV_MAG_APER2 |   |
| NUV_MAG_ERR_APER2 |   |
| NUV_MASK_FLAG_APER2 |   |
| NUV_EDGE_FLAG_APER2 |   |
| NUV_SUM_APER3 |   |
| NUV_EDGE_APER3 |   |
| NUV_MASK_APER3 |   |
| NUV_CPS_APER3 |   |
| NUV_CPS_ERR_APER3 |   |
| NUV_MAG_APER3 |   |
| NUV_MAG_ERR_APER3 |   |
| NUV_MASK_FLAG_APER3 |   |
| NUV_EDGE_FLAG_APER3 |   |
| NUV_SUM_APER4 |   |
| NUV_EDGE_APER4 | 